In [1]:
import numpy as np
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf
import os

In [2]:
data = np.load('/home/ML4NO/Data/n1000000_0910_all_flat.npz')
model_index = 1
model = tf.keras.models.load_model('/home/ML4NO/ML_DUNE/Regression/models/0910_theta23_{}.h5'.format(model_index))
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 264)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 264)               0         
_________________________________________________________________
normalization (Normalization (None, 264)               529       
_________________________________________________________________
dense (Dense)                (None, 512)               135680    
_________________________________________________________________
re_lu (ReLU)                 (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
re_lu_1 (ReLU)               (None, 512)               0     

In [3]:
data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
target = data['theta23']

x_train = data_all[:10000]
y_train = target[:10000]
x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]
x_test = data_all[900000:]
y_test = target[900000:]

In [5]:
scale_steps = np.logspace(-3, 0, 30)
before_train_loss = []
after_train_loss = []

for scale in scale_steps:
    x_train2_gen = np.random.normal(x_train2, np.sqrt(x_train2)*scale)
    x_test_gen = np.random.normal(x_test, np.sqrt(x_test)*scale)

    before_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

    model.fit(x_train2_gen, y_train2,
               validation_split = 0.1,
               batch_size=64,
               epochs=1,
               verbose=1,
               shuffle = True
             )

    after_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

12515/12516 [============================>.] - ETA: 0s - loss: 0.0047 - mean_squared_error: 0.0047

KeyboardInterrupt: 

In [5]:
index = 1
path = '/home/ML4NO/ML_DUNE/Regression/models_furthurTrain/0910_theta23_{}_{}.h5'
while os.path.isfile(path.format(model_index, index)):
    index += 1
model.save(path.format(model_index, index))
outfile = {'scale_steps': scale_steps,
           'before_train_loss': before_train_loss,
           'after_train_loss' :after_train_loss}
np.save(file = '/home/ML4NO/ML_DUNE/Regression/models_furthurTrain/0910_theta23_{}_{}_result.npy'.format(model_index, index),
        arr = outfile)